In [ ]:
pip install -r requirements.txt || true

In [ ]:
import pandas as pd
import requests as rq
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path

# Демонстрация того, как будет происходить процесс совмещения на двух днях

In [ ]:
# Ссылка на данные
url1 = "https://iss.moex.com/iss/analyticalproducts/futoi/securities/si.json?from=2023-11-07&till=2023-11-08&table_type=full"
url2 = "https://iss.moex.com/iss/analyticalproducts/futoi/securities/si.json?from=2023-11-06&till=2023-11-07&table_type=full"

In [ ]:
# Отправляем GET-запрос по указанной ссылке
response = rq.get(url1)

# Проверяем статус ответа
if response.status_code == 200:
    # Загружаем данные из ответа в переменную data в случае успеха
    data = response.json()
else:
    print("Ошибка при получении данных")
    

columns = response.json()["futoi"]['columns']

data = response.json()["futoi"]["data"]

df1 = pd.DataFrame(data, columns=columns)


# Отправляем GET-запрос по указанной ссылке
response = rq.get(url2)

# Проверяем статус ответа
if response.status_code == 200:
    # Загружаем данные из ответа в переменную data в случае успеха
    data = response.json()
else:
    print("Ошибка при получении данных")


columns = response.json()["futoi"]['columns']

data = response.json()["futoi"]["data"]

df2 = pd.DataFrame(data, columns=columns)

In [ ]:
print(df1.sort_values(by=['ticker', 'tradedate','tradetime']).reset_index().drop('index', axis=1))
print(len(np.array(df1['ticker'])))

In [ ]:
df2.sort_values(by=['ticker', 'tradedate', 'tradetime']).reset_index().drop('index', axis=1)

In [ ]:
print(len(np.array(df2['ticker'])))

In [ ]:
frames = [df1, df2]

result = pd.concat(frames).sort_values(by=['ticker', 'tradedate', 'tradetime']).reset_index().drop('index', axis=1)
filename = "test_date.csv"
result.to_csv(filename, sep=',', index=False, encoding='utf-8')

print(result)

print(len(np.array(result['ticker'])))

Далее будет пробегать по массиву дат и таким образом сформируем всю таблицу

Ниже я задаю массив всех дат

# Парсинг данных о позиций

Сначала мы создаём массив, который содержить все даты, за которые мы хотим получить данные

In [ ]:
start_date = datetime(2020, 5, 8)
end_date = datetime.now()

current_date = start_date
array_date = []
while current_date <= end_date:
    array_date += [current_date.strftime("%Y-%m-%d")]
    current_date += timedelta(days=1)

Тут мы формируем массив из дней, а затем, если сервер работает хорошо, то возвращаем данные за весь промежуток от 5 августа 2020 до сегодняшнего дня. 

Если сервер перестаёт отвечать, тогда полученные данные мы собираем в файл и при повторном запуске программа проверить наши файлы и начнёт с последнего записанного дня.

В качестве отладки печатается дата дня, который мы сейчас получаем. 

In [ ]:
def add_new_day(date_today: str, ticker_name: str):
    date_yesterday = (datetime.strptime(date_today, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://iss.moex.com/iss/analyticalproducts/futoi/securities/{ticker_name}.json?from={date_today}&till={date_today}&table_type=full"

    while True:
        try:
            # Ваш код, который может вызвать ошибку
            response = rq.get(url)
            # break  # Если ошибки не возникло, выходим из цикла
        except Exception as e:
            # Обработка других исключений
            print(f"Произошла ошибка: {e}")
        else:
            break

    # Проверяем статус ответа
    if response.status_code != 200:
        print("Ошибка при получении данных")
        add_new_day(date_today)

    df_for_add = pd.DataFrame(response.json()["futoi"]["data"], columns=response.json()["futoi"]['columns'])
    df_for_add.sort_values(by=['ticker', 'tradedate','tradetime']).reset_index().drop('index', axis=1)
   
    return df_for_add

# ticker_names=['si', 'br', 'ri', 'eu', 'mx', 'sr', 'ox', 'gz', 'yn', 'fv', 'vi', 'lk']
ticker_names=['sr', 'gz', 'lk', 'vb', 'rn', 'mn', 'af', 'al', 'sn', 'yn', 'tt', 'nm', 'hy', 'me', 'fv', 'gk', 'mg', 'ml']

for ticker_name in ticker_names:
    if Path(f"{ticker_name}_full_date.csv").exists():
        df = pd.read_csv(f"{ticker_name}_full_date.csv")
        last_valid_date_index = df['tradedate'].last_valid_index()
        last_valid_date_value  = df['tradedate'][last_valid_date_index]
        date_tomorrow = (datetime.strptime(last_valid_date_value, '%Y-%m-%d') + timedelta(days=1)).strftime("%Y-%m-%d")
        print(date_tomorrow)
        if(not date_tomorrow in array_date):
            pass
        else:
            df_array = [df]
            for date in array_date[array_date.index(date_tomorrow)+1:]:
                df_array += [add_new_day(date, ticker_name)]
                print(date)
            df = pd.concat(df_array).sort_values(by=['ticker', 'tradedate', 'tradetime']).reset_index().drop('index', axis=1)
            print(df)
            filename = f"{ticker_name}_full_date.csv"
            df.to_csv(filename, sep=',', index=False, encoding='utf-8')
    else:
        print(f"Файл '{ticker_name}_full_date.csv' не существует.")
        df_array = []
        for date in array_date:
            df_array += [add_new_day(date, ticker_name)]
            print(date)
        df = pd.concat(df_array).sort_values(by=['ticker', 'tradedate', 'tradetime']).reset_index().drop('index', axis=1)
        print(df)
        filename = f"{ticker_name}_full_date.csv"
        df.to_csv(filename, sep=',', index=False, encoding='utf-8')

    

Удаление повторяющихся данных (в силу того, как МосБиржа возвращала данные (в выходные дни вместо пустого запроса она возвращает последнюю цену за последний рабочий день, то есть 14*20 одинаковых записей, в силу чего эта ячейка позволяла сэкономить, примерно, 300 МБ))

In [ ]:
# ticker_names=['si', 'br', 'ri', 'eu', 'mx', 'sr', 'ox', 'gz', 'yn', 'fv', 'vi', 'lk']

for ticker_name in ticker_names:
    your_file = f"{ticker_name}_full_date.csv" 
# Загрузка данных
    df = pd.read_csv(your_file)

# Удаление повторяющихся строк
    unique_rows = df.drop_duplicates()

# Сохранение результата
    unique_rows.to_csv(your_file, index=False)

Итоговые данные не идеальны, поэтому мы их будем обрабатывать в других данных.